In [1]:
import json
import os
import re
from datetime import datetime, timedelta
from urllib.parse import urlparse
import time
import yaml

import pandas as pd 

import sqlite3

import pyperclip

import bs4
from bs4 import BeautifulSoup
import requests

import dotenv

import IPython
from IPython.display import HTML, Markdown, display

from atproto import Client

import PIL
from PIL import Image

# disable 'show more outputs' nonsense
from IPython.core.interactiveshell import InteractiveShell

# Always display all outputs in a cell
InteractiveShell.ast_node_interactivity = "all"


In [2]:
print(f"requests        {requests.__version__}")
print(f"BeautifulSoup   {bs4.__version__}")


requests        2.32.3
BeautifulSoup   4.13.3


In [3]:
dotenv.load_dotenv()


True

# Load posts from BlueSky and format for Substack or a blog post
for now I share the interesting stuff on bluesky and then use this code to grab latest BlueSky 'tweets' and format a [Substack post](https://skynetandchill.com)


In [4]:
client = Client(base_url='https://bsky.social')
client.login(os.environ['BSKY_USERNAME'], os.environ['BSKY_SECRET'])

mydid = {"did":"did:plc:qomkdnxrqw3gkbytdxea5z65"}

data = client.get_author_feed(
    actor=mydid['did'],
    filter='posts_and_author_threads',
    limit=80,
)


ProfileViewDetailed(did='did:plc:qomkdnxrqw3gkbytdxea5z65', handle='skynetandchill.com', associated=ProfileAssociated(chat=ProfileAssociatedChat(allow_incoming='following', py_type='app.bsky.actor.defs#profileAssociatedChat'), feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:qomkdnxrqw3gkbytdxea5z65/bafkreihi55ddvmohg6dztiwlkwdbts5fd76iaenef2sriacvj7ibtezagi@jpeg', banner='https://cdn.bsky.app/img/banner/plain/did:plc:qomkdnxrqw3gkbytdxea5z65/bafkreigu723bmkba3mff4krhcrrf7b5zw2svixwbvtj54v5ho64o2xpmzu@jpeg', created_at='2024-02-12T00:15:55.099Z', description='Daily artificial intelligence news, lovingly curated by man and machine. https://www.skynetandchill.com\n\nNeo-Luddite AI maven. On a long enough timeline, p(doom) for everything goes to 1.', display_name='SkynetAndChill.com', followers_count=330, follows_count=347, indexed_at='2024-11-21T15:40:08.470Z', joined_via_starter_p

In [5]:
def remove_urls(text):
    # Regular expression to match URLs
    url_pattern = r'https?://\S+|www\.\S+'
    # Substitute found URLs with an empty string
    clean_text = re.sub(url_pattern, '', text)
    return clean_text


In [6]:
def rawfetchurl(url, timeout=60):
    """get url using requests with specified timeout. return response object, status, content-type"""
    try:
        response = requests.get(url, timeout=timeout)
    except httplib.BadStatusLine:
        log("Bad response (?) fetching url %s " % url)
        response = None
    except requests.Timeout:
        log("Timeout fetching url %s " % url)
        response = None
    except requests.ConnectionError as e:
        log("Connection error (%s) fetching url %s " % (str(e), url))
        response = None
    except requests.TooManyRedirects:
        log("Too many redirects fetching url %s " % url)
        response = None
    except requests.exceptions.MissingSchema:
        log("Missing schema url %s " % url)
        response = None
    except requests.exceptions.InvalidSchema:
        log("Invalid schema url %s " % url)
        response = None
    except requests.exceptions.InvalidURL as e:
        log("Invalid url %s, %s" % (url, str(e)))
        response = None
    except ValueError as e:
        # don't log url because possibly malformed url
        log("ValueError, url ?: ? ")
        response = None
    except httplib.IncompleteRead as e:
        log("IncompleteRead, url %s: %s " % (url, str(e)))
        response = None
    except urllib3.exceptions.SSLError as e:
        log("SSLError, url %s: %s " % (url, str(e)))
        response = None
    except requests.exceptions.ContentDecodingError as e:
        log("SSLError, url %s: %s " % (url, str(e)))
        response = None
    except requests.exceptions.ChunkedEncodingError as e:
        log("ChunkedEncodingError, url %s: %s " % (url, str(e)))
        response = None
    except UnicodeEncodeError as e:
        log("UnicodeEncodeError, url %s: %s " % (url, str(e)))
        response = None
    except OpenSSL.SSL.SysCallError as e:
        log("OpenSSL.SSL.SysCallError, url %s: %s " % (url, str(e)))
        response = -1
    except OpenSSL.SSL.ZeroReturnError as e:
        log("OpenSSL.SSL.ZeroReturnError, url %s: %s " % (url, str(e)))
        response = -1

    # except requests.packages.urllib3.exceptions.DecodeError as e:
    #     utilLog("DecodeError, url %s: %s " % (url, str(e)))
    #     response = None

    return response



In [7]:
def resize_and_crop(input_image_path, output_image_path, desired_height=240):
    # Load the image
    with Image.open(input_image_path) as img:
        img = img.convert('RGB')

        # Calculate the new width maintaining the aspect ratio
        aspect_ratio = img.width / img.height
        new_width = int(desired_height * aspect_ratio)

        # Resize the image
        resized_img = img.resize((new_width, desired_height))

        # Save the resized image
        resized_img.save(output_image_path)


In [8]:
# attempt to remove traiing inline URLs

def truncate_last_occurrence(text: str) -> str:
    # Find trailing occurrence of a space followed by any sequence of characters followed by 3 periods
    pattern = r'\s+\S+\.{3}$'
    return re.sub(pattern, '', text)

# Example text for testing

example_text = """Elon Musk says we'll run out of power capacity to run all the AI chips in 2025
newatlas.com/technology/e..."""

# Truncate the last occurrence
print(truncate_last_occurrence(example_text))


Elon Musk says we'll run out of power capacity to run all the AI chips in 2025


In [9]:
def get_og_tags(url):
    """get a dict of Open Graph og: tags such as title in the HEAD of a URL"""
    retdict = {}
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            head = soup.head
            if head:
                og_tags = head.find_all(
                    property=lambda prop: prop and prop.startswith("og:")
                )
                for tag in og_tags:
                    if "content" in tag.attrs:
                        retdict[tag["property"]] = tag["content"]

                page_title = ""
                title_tag = soup.find("title")
                if title_tag:
                    page_title = title_tag.text
                    if page_title:
                        retdict["title"] = page_title
        return retdict
    except requests.RequestException as e:
        log(f"Error fetching {url}: {e}")
    return retdict


url = "https://druce.ai"
get_og_tags(url)

{'og:site_name': 'Druce.ai',
 'og:title': 'Druce.ai',
 'og:type': 'website',
 'og:description': "Druce's Blog on Machine Learning, Tech, Markets and Economics",
 'og:url': 'https://druce.ai/',
 'title': 'Druce.ai'}

In [10]:
def delete_files(outputdir):

    # Iterate over all files in the directory
    for filename in os.listdir(outputdir):
        if filename.startswith('.'):
            continue
        file_path = os.path.join(outputdir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.remove(file_path)  # Remove the file
            elif os.path.isdir(file_path):
                # If you want to remove subdirectories as well, use os.rmdir() here
                pass
        except Exception as e:
            log(f'Failed to delete {file_path}. Reason: {e}')
            

In [11]:
# import sqlite3
# import tldextract

# # Path to your SQLite database
# db_path = "articles.db"

# # Connect to the database
# conn = sqlite3.connect(db_path)
# cur = conn.cursor()

# # Fetch all rows (id, hostname)
# cur.execute("SELECT id, hostname FROM sites")
# rows = cur.fetchall()

# for row in rows:
#     site_id, hostname = row
#     # Extract the registered domain (e.g., 'bbc.co.uk', '9to5mac.com')
#     extracted = tldextract.extract(hostname)
#     if extracted.domain and extracted.suffix:
#         registered_domain = f"{extracted.domain}.{extracted.suffix}"
#         # Update the domain column
#         print(f"{hostname} -> {registered_domain}")
#         cur.execute(
#             "UPDATE sites SET domain = ? WHERE id = ?",
#             (registered_domain, site_id))
#     else:
#         registered_domain = None  # fallback if parsing fails

# # Commit changes and close
# conn.commit()
# conn.close()

In [12]:
imgdir = 'tmp'  # for images
delete_files(imgdir)

conn = sqlite3.connect('articles.db')
query = "select * from sites"
sites_df = pd.read_sql_query(query, conn)
sites_dict = {row.hostname: row.site_name for row in sites_df.itertuples()}
conn.close()


In [13]:
def parse_feed():
    display_dict = {}
    image_dict = {}
    for i, post in enumerate(data.feed):
        
        # check date
        date_str = data.feed[0].post.record.created_at
        dt = datetime.fromisoformat(date_str.rstrip('Z'))
        # break if more than 30 hours ago
        if dt  < datetime.now() - timedelta(hours=30):
            break

        # get post_str, post_url
        post_str = post.post.record.text.rstrip()
        post_str = truncate_last_occurrence(post_str)
        post_str = post_str.replace("$", "\\\$")  # so Markdown doesn't interpret $ as latex escape

        try:
            post_url = post.post.record.embed.external.uri.rstrip()
        except:
            post_url = ""

        if post_url:
            # get site name , image using og tags
            try:
                tag_dict = get_og_tags(post_url)
            except:
                tag_dict = {}

            site_name = tag_dict.get('og:site_name')
            img_url = tag_dict.get('og:image')

            # get site_name from stored values as backup
            if not(site_name):
                parsed_url = urlparse(post_url)
                hostname = parsed_url.hostname
                site_name = sites_dict.get(hostname, hostname)

            md_str = f"[{post_str}]({post_url})  - {site_name}"

            display_dict[i]=md_str

            # write local copy of image
            if img_url:
                try:
                    r = rawfetchurl(img_url)
                    content_type = r.headers['Content-Type']
                    content_type = content_type[content_type.find('/')+1:]
                    impath = f"{imgdir}/source{i}.{content_type}"
                    with open(impath, 'wb') as file:
                        file.write(r.content)
                    output_path = f'{imgdir}/Image{i}.jpg'
                    resize_and_crop(impath, output_path)
                    image_dict[i] = f"![image]({output_path})\n\n"
                except Exception as e:
                    print(e)

        else: # no url
            md_str = f"\n{post_str}" + "\n\n"
            display_dict[i]=md_str
            
    return display_dict, image_dict

display_dict, image_dict = parse_feed()
len(display_dict)


cannot identify image file 'tmp/source47.html; charset=utf-8'


80

In [15]:
output_str = ""
for i in sorted(display_dict.keys()):
    md_str = f"{i}. "
    md_str += display_dict.get(i)
    if image_dict.get(i):
        md_str += image_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str

output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

display(Markdown(output_str))


0. [Sam Altman and Jony Ive’s ChatGPT device is probably going to look like an iPod Shuffle you can wear around your neck - report reveals more about the hyped AI hardware](https://www.techradar.com/computing/artificial-intelligence/sam-altman-and-jony-ives-chatgpt-device-is-probably-going-to-look-like-an-ipod-shuffle-you-can-wear-around-your-neck-report-reveals-more-about-the-hyped-ai-hardware)  - TechRadar![image](tmp/Image0.jpg)



---

1. [Business owners are using AI-generated "concerned residents" to fight a proposed bus lane](https://torontolife.com/city/bathurst-business-owners-are-using-ai-generated-concerned-residents-to-fight-a-proposed-bus-lane/)  - torontolife.com

---

2. [China challenges for the lead in the race to self-driving vehicles](https://www.ft.com/content/41bac535-9237-444b-ba56-a9cf38c11b3f)  - Financial Times![image](tmp/Image2.jpg)



---

3. [Microsoft CEO Satya Nadella on the Future of AI](https://www.youtube.com/watch?v=w87UvmMcmW4)  - YouTube![image](tmp/Image3.jpg)



---

4. [Generative AI models are skilled in the art of bullshit](https://www.ft.com/content/55c08fc8-2f0b-4233-b1c6-c1e19d99990f)  - Financial Times![image](tmp/Image4.jpg)



---

5. [What ‘Sovereign AI’ Means for Chip Makers](https://www.wsj.com/finance/stocks/nvidia-ai-chipmakers-country-deals-05b5f44d)  - The Wall Street Journal

---

6. [Usuing dataviz and AI to map Musk's ideological journey](https://www.wsj.com/tech/elon-musk-politics-trump-social-media-267d34c8)  - The Wall Street Journal

---

7. [Civitai is removing all models and images depicting real-world individuals from its platform, including public figures, celebrities, influencers, private individuals, and fan-art based on real persons.](https://civitai.com/articles/15022/policy-update-removal-of-real-person-likeness-content)  - civitai.com![image](tmp/Image7.jpg)



---

8. [Reddit user tells ChatGPT about silly business idea he talked about in his sleep, ChatGPT says it's brilliant but ultimately says, do not quit your day job](https://www.reddit.com/r/ChatGPT/comments/1ksqog2/i_told_chatgpt_i_was_going_to_quit_my_job_to/)  - Reddit

---

9. [Research on benchmarks shows that every LLM is a sycophantic simp to its users, with GPT-4o highest, and Gemini-1.5-Flash the lowest.](https://venturebeat.com/ai/after-gpt-4o-backlash-researchers-benchmark-models-on-moral-endorsement-find-sycophancy-persists-across-the-board/)  - VentureBeat![image](tmp/Image9.jpg)



---

10. [Tech CEOs are using AI to replace themselves](https://www.theverge.com/news/673194/tech-ceos-zoom-klarna-replace-earnings)  - The Verge![image](tmp/Image10.jpg)



---

11. [How to start a \\$1b one-person unicorn](https://www.zdnet.com/article/first-1b-business-with-one-human-employee-will-happen-in-2026-says-anthropic-ceo/)  - ZDNET![image](tmp/Image11.jpg)



---

12. [Demis Hassabis, The Man Who ‘A.G.I.-Pilled’ Google (and who Musk called a supervillain per Karen Hao)](https://www.nytimes.com/2025/05/23/podcasts/google-ai-demis-hassabis-hard-fork.html)  - The New York Times

---

13. [Longtime Apple analyst says OpenAI/Jony Ive tie-up might pose a serious challenge to Apple. (Ive isn't joining himself, but his company will drive design.)](https://www.benzinga.com/markets/equities/25/05/45560144/apple-faces-first-serious-threat-in-20-years-as-openai-acquires-iphone-makers-former-design-chie?ref=biztoc.com)  - Benzinga![image](tmp/Image13.jpg)



---

14. [Claude AI, given access to emails that outed an engineer's affair, and also intent to deactivate the AI, blackmailed the engineer 84% of the time. (Something to think about before giving Google AI access to all your email and chat histories)](https://www.businessinsider.com/claude-blackmail-engineer-having-affair-survive-test-anthropic-opus-2025-5)  - Business Insider![image](tmp/Image14.jpg)



---

15. [Bombing AI crawlers with a high volume of PR spam that wouldn't fool reporters could probably fool AI](https://www.fastcompany.com/91338790/ai-could-supercharge-go-direct-pr-media)  - Fast Company

---

16. [AI-assisted transcription sped up clinical documentation by 60% and improved patient consultation record accuracy.](https://emj.bmj.com/content/42/6/e1.1)  - Emergency Medicine Journal![image](tmp/Image16.jpg)



---

17. [Wheeled, rugged robot dog built for industrial missions
in extreme environments](https://www.foxnews.com/tech/wheeled-rugged-robot-dog-built-extreme-industrial-missions)  - Fox News![image](tmp/Image17.jpg)



---

18. [Apple plans AI glasses for 2026, nixes watch with camera, camera on AirPods still in progress.](https://www.bloomberg.com/news/articles/2025-05-22/apple-plans-glasses-for-2026-as-part-of-ai-push-nixes-watch-with-camera)  - Bloomberg

---

19. [Maybe if you need AI to determine the 'plain meaning' of a statute, then the meaning is not that plain.](https://www.abajournal.com/voice/article/navigating-the-ai-revolution-in-law-school-lessons-from-the-front-lines)  - www.abajournal.com

---

20. [After Klarna, Zoom's CEO also uses an AI avatar on quarterly call. (Makes more sense if you're offering your own avatars as a videoconferencing company)](https://techcrunch.com/2025/05/22/after-klarna-zooms-ceo-also-uses-an-ai-avatar-on-quarterly-call/)  - TechCrunch![image](tmp/Image20.jpg)



---

21. [I let Google's Jules AI agent into my code repo and it did four hours of work in an instant](https://www.zdnet.com/article/i-let-googles-jules-ai-agent-into-my-code-repo-and-it-did-four-hours-of-work-in-an-instant/)  - ZDNET![image](tmp/Image21.jpg)



---

22. [Google’s Veo 3 Is Already Deepfaking All of YouTube’s Most Smooth-Brained Content](https://gizmodo.com/googles-veo-3-is-already-deepfaking-all-of-youtubes-most-smooth-brained-content-2000606144)  - Gizmodo![image](tmp/Image22.jpg)



---

23. [AI Shows Higher Emotional IQ than Humans](https://neurosciencenews.com/ai-llm-emotional-iq-29119/)  - Neuroscience News![image](tmp/Image23.jpg)



---

24. [Human, all-too human](https://theconversation.com/evidence-shows-ai-systems-are-already-too-much-like-humans-will-that-be-a-problem-256980)  - The Conversation![image](tmp/Image24.jpg)



---

25. [Anthropic releases Opus 4 under stricter safety measures than any prior model after tests showed it could potentially aid novices in making biological weapons](https://time.com/7287806/anthropic-claude-4-opus-safety-bio-risk/)  - TIME![image](tmp/Image25.jpg)



---

26. [OpenAI Announces Stargate Data Center Expansion in Abu Dhabi](https://www.bloomberg.com/news/articles/2025-05-22/openai-announces-stargate-data-center-expansion-in-abu-dhabi)  - Bloomberg

---

27. ["I used to make \\$500k Pharmaceutical commercial ads, but now I made this for \\$500 in Veo 3."](https://www.reddit.com/r/ChatGPT/comments/1ksljfs/i_used_to_make_500k_pharmaceutical_commercial_ads/)  - Reddit

---

28. [In lawsuit over teen's death, judge rejects arguments that AI chatbots have free speech rights](https://financialpost.com/pmn/in-lawsuit-over-teens-death-judge-rejects-arguments-that-ai-chatbots-have-free-speech-rights)  - financialpost.com

---

29. [Anthropic Claude 4 models a little more willing than before to blackmail some users. Open the pod bay door.](https://www.theregister.com/2025/05/22/anthropic_claude_opus_4_sonnet/)  - The Register

---

30. [Given computer tools, Claude 4 Opus autonomously tries to contact authorities, press if it thinks you’re doing something ‘egregiously immoral’.](https://venturebeat.com/ai/anthropic-faces-backlash-to-claude-4-opus-behavior-that-contacts-authorities-press-if-it-thinks-youre-doing-something-immoral/)  - VentureBeat![image](tmp/Image30.jpg)



---

31. [we are cooked](https://www.reddit.com/r/singularity/comments/1ksj9dm/its_over_were_cooked_says_girl_that_literally/)  - Reddit

---

32. [Anthropic's new AI model turns to blackmail when engineers try to take it offline](https://techcrunch.com/2025/05/22/anthropics-new-ai-model-turns-to-blackmail-when-engineers-try-to-take-it-offline/)  - TechCrunch![image](tmp/Image32.jpg)



---

33. [this pisses me off](https://support.mozilla.org/en-US/kb/future-of-pocket)  - support.mozilla.org

---

34. [The Justice Department is probing whether Alphabet Inc.’s Google violated antitrust law with an agreement to obtain the artificial intelligence technology of a Character.ai without acquiring it.](https://www.bloomberg.com/news/articles/2025-05-22/google-faces-antitrust-investigation-over-deal-for-ai-fueled-chatbot-technology)  - Bloomberg

---

35. [Anthropic released Claude 4 Sonnet and Opus, with really long thinking time](https://arstechnica.com/ai/2025/05/anthropic-calls-new-claude-4-worlds-best-ai-coding-model/)  - Ars Technica![image](tmp/Image35.jpg)



---

36. [Melania Trump releases audiobook of her memoir ‘created entirely’ with AI. In fact the version of her you see at the White House and accompanying Trump is also 100% a holograph created by AI.](https://thehill.com/blogs/in-the-know/5313558-melania-trump-ai-memoir-audiobook/)  - The Hill

---

37. [Politico’s Newsroom Is Starting a Legal Battle With Management Over AI](https://www.wired.com/story/politico-workers-axel-springer-artificial-intelligence/)  - WIRED![image](tmp/Image37.jpg)



---

38. [OpenAI’s Ambitions Just Became Crystal Clear](https://www.theatlantic.com/technology/archive/2025/05/openai-io-jony-ive/682884/)  - The Atlantic![image](tmp/Image38.jpg)



---

39. [Teens are sexting with AI. Here’s what parents should know.](https://www.washingtonpost.com/technology/2025/05/21/teens-sexting-ai-chatbots-parents/)  - The Washington Post

---

40. [The folly of Trump’s gulf states AI chip deals, which will reshape the future of AI infrastructure — and who will control it.](https://www.washingtonpost.com/opinions/2025/05/22/trump-ai-deals-gulf-states/)  - The Washington Post

---

41. [Glama MCP directory](https://glama.ai/mcp/servers)  - GlamAI

---

42. [Gemini Live on iPhone can outshine advanced voice assistant, when it can see your screen as well as surroundings](https://www.techradar.com/computing/artificial-intelligence/iphones-just-got-googles-best-ai-feature-for-free-and-it-could-genuinely-make-me-switch-back-from-android)  - TechRadar![image](tmp/Image42.jpg)



---

43. [A mathematical proof that algorithms can trade memory for time, like with enough memory you can sort in O(1) with a bucket sort.](https://www.quantamagazine.org/for-algorithms-a-little-memory-outweighs-a-lot-of-time-20250521/)  - Quanta Magazine![image](tmp/Image43.jpg)



---

44. [AI video characters who don't want to be in your narrative are a bit disturbing](https://www.reddit.com/r/ChatGPT/comments/1kscj2a/selfaware_ai_characters_made_with_veo_3/)  - Reddit

---

45. [Sergey Brin said he has used AI for leadership tasks at Google, including delegating tasks and even finding someone to promote.](https://www.businessinsider.com/sergey-brin-uses-ai-management-leadership-summaries-google-gemini-2025-5)  - Business Insider![image](tmp/Image45.jpg)



---

46. [Tantalizing details of Jony Ive's AI device leak after OpenAI meeting](https://appleinsider.com/articles/25/05/22/tantalizing-details-of-jony-ives-ai-device-leak-after-openai-meeting)  - AppleInsider![image](tmp/Image46.jpg)



---

47. [TRICORDER AI-enabled stethoscope FTW](https://bmjopen.bmj.com/content/15/5/e098030)  - BMJ Open

---

48. [When an agent screws up, who is responsible, and liable? Developers/platforms, the person who unleashed it?](https://www.wired.com/story/ai-agents-legal-liability-issues/)  - WIRED![image](tmp/Image48.jpg)



---

49. [Microsoft put ex-Facebook exec Jay Parikh in charge of a 'Core AI' unit building agents everywhere in the MS ecosystem](https://www.businessinsider.com/memos-how-satya-nadellas-ai-czar-plans-to-change-microsoft-2025-5)  - Business Insider![image](tmp/Image49.jpg)



---

50. [Mistral AI launches Devstral, powerful new open source SWE agent model that runs on laptops](https://venturebeat.com/ai/mistral-ai-launches-devstral-powerful-new-open-source-swe-agent-model-that-runs-on-laptops/)  - VentureBeat![image](tmp/Image50.jpg)



---

51. [Kayak Pivots From Travel Search to AI-Driven Travel Agent](https://www.pymnts.com/news/artificial-intelligence/2025/kayak-pivots-from-search-to-ai-driven-travel-assistant/)  - PYMNTS.com![image](tmp/Image51.jpg)



---

52. [Vast Data says it has built an AI- and data-first operating system for large-scale agent deployment on the edge](https://siliconangle.com/2025/05/21/vast-data-says-built-operating-system-large-scale-agent-deployment/)  - SiliconANGLE![image](tmp/Image52.jpg)



---

53. [Two timely and myth-busting books show that the AI boom is driven by the pursuit of power, wealth and hubris](https://www.ft.com/content/1721a7e3-8fd6-47e7-83ce-e9e34e1db6eb)  - Financial Times![image](tmp/Image53.jpg)



---

54. [Pitting the new Google Imagen against ChatGPT for AI image generation, Google might be better at edits without completely changing the original, ChatGPT might be better at text and style edits like Ghiblification](https://www.techradar.com/computing/artificial-intelligence/i-pitched-the-new-google-gemini-against-chatgpt-for-ai-image-generation-and-the-results-shocked-me)  - TechRadar![image](tmp/Image54.jpg)



---

55. [Simon Willison: Building software on top of Large Language Models](https://simonwillison.net/2025/May/15/building-on-llms/)  - Simon Willison’s Weblog![image](tmp/Image55.jpg)



---

56. [How AI is transforming drug discovery: AlphaFold, AI to identify drug candidates, robots in wet labs, AI in clinical testing](https://www.news-medical.net/health/Why-Drug-Discovery-Needs-Robots-and-Artificial-Intelligence.aspx)  - News-Medical![image](tmp/Image56.jpg)



---

57. [Researchers use AI to target gene therapies to brain cells using viruses](https://neurosciencenews.com/ai-genetics-brain-spine-29903/)  - Neuroscience News![image](tmp/Image57.jpg)



---

58. [Gemini Diffusion was the sleeper hit of Google I/O and some say its blazing speed could reshape the AI model wars](https://tech.yahoo.com/articles/gemini-diffusion-sleeper-hit-google-215427722.html)  - Yahoo Tech![image](tmp/Image58.jpg)



---

59. [Klarna used an AI avatar of its CEO to deliver earnings, it said](https://techcrunch.com/2025/05/21/klarna-used-an-ai-avatar-of-its-ceo-to-deliver-earnings-it-said/)  - TechCrunch![image](tmp/Image59.jpg)



---

60. [What Sam Altman Told OpenAI About the Secret Device He’s Making With Jony Ive](https://www.wsj.com/tech/ai/what-sam-altman-told-openai-about-the-secret-device-hes-making-with-jony-ive-f1384005?st=RUGPjK&reflink=desktopwebshare_permalink)  - The Wall Street Journal

---

61. [Google has a big AI advantage: it already knows everything about you](https://www.theverge.com/tech/671201/google-personal-context-ai-advantage-data)  - The Verge![image](tmp/Image61.jpg)



---

62. [Microsoft blocks emails that contain ‘Palestine’, 'Gaza', 'genocide' after employee protests.](https://www.theverge.com/tech/672312/microsoft-block-palestine-gaza-email)  - The Verge![image](tmp/Image62.jpg)



---

63. [Demis Hassabis and Sergey Brin on AI Scaling, AGI Timeline, Robotics, Simulation Theory](https://www.bigtechnology.com/p/demis-hassabis-and-sergey-brin-on)  - www.bigtechnology.com![image](tmp/Image63.jpg)



---

64. [How Peter Thiel’s Relationship With Eliezer Yudkowsky Launched the AI Revolution](https://www.wired.com/story/book-excerpt-the-optimist-open-ai-sam-altman/)  - WIRED![image](tmp/Image64.jpg)



---

65. [Orson Welles Can Now Be Your AI-Generated Tour Guide](https://variety.com/2025/digital/news/orson-welles-ai-voice-storyrabbit-audio-storytelling-app-1236406112/)  - Variety![image](tmp/Image65.jpg)



---

66. [What is still worth learning in the age of AI?](https://www.the-independent.com/tech/ai-education-learning-agi-chatgpt-b2750767.html)  - www.the-independent.com

---

67. [What AI Thinks It Knows About You](https://www.theatlantic.com/technology/archive/2025/05/inside-the-ai-black-box/682853/)  - The Atlantic![image](tmp/Image67.jpg)



---

68. [The enterprise AI trough of disillusionment](https://www.economist.com/business/2025/05/21/welcome-to-the-ai-trough-of-disillusionment)  - The Economist![image](tmp/Image68.jpg)



---

69. [Sam & Jony

Elon and Sam each want to be the new Steve Jobs but Sundar and Satya and maybe others might have something to say about it.](https://www.youtube.com/watch?v=W09bIpc_3ms)  - YouTube![image](tmp/Image69.jpg)



---

70. [There should really be an opt-out over ChatGPT's persistent memory.](https://simonwillison.net/2025/May/21/chatgpt-new-memory/)  - Simon Willison’s Weblog![image](tmp/Image70.jpg)



---

71. [Foxconn says AI and robotics will take over assembly lines](https://9to5mac.com/2025/05/21/foxxconn-ai-and-robotics-replace-humans/)  - 9to5Mac![image](tmp/Image71.jpg)



---

72. [Sam Altman says 'humanoid robots moment' is not far away  and the world isn't ready.](https://www.businessinsider.com/sam-altman-openai-humanoid-robots-future-of-work-ai-2025-5)  - Business Insider![image](tmp/Image72.jpg)



---

73. [News publishers call Google’s AI Mode ‘theft’](https://www.theverge.com/news/672132/news-media-alliance-google-ai-mode-theft)  - The Verge![image](tmp/Image73.jpg)



---

74. [Microsoft Aurora AI model beats global agencies on cyclone track forecasts](https://www.theregister.com/2025/05/21/earth_system_model_hurricane_forecast/)  - The Register

---

75. [Chatbot Arena Raises \\$100 Million for Platform That Compares AI Models](https://www.pymnts.com/news/artificial-intelligence/2025/chatbot-arena-raises-100-million-dollars-platform-compares-ai-models/)  - PYMNTS.com![image](tmp/Image75.jpg)



---

76. [OpenAI updates Responses API with MCP support, image generation, and more](https://openai.com/index/new-tools-and-features-in-the-responses-api/)  - OpenAI

---

77. [OpenAI Stargate data center in Texas gets \\$11.6 billion in funding commitments](https://www.pymnts.com/news/artificial-intelligence/2025/openai-texas-data-center-lands-11-6-billion-in-funding-commitments/)  - PYMNTS.com![image](tmp/Image77.jpg)



---

78. [Abu Dhabi’s MGX, Nvidia to build 1.4GW AI data center campus near Paris by 2028](https://www.semafor.com/article/05/21/2025/abu-dhabis-mgx-nvidia-to-build-europes-largest-ai-campus-in-paris)  - Semafor![image](tmp/Image78.jpg)



---

79. [A Veo3 compilation](https://www.reddit.com/r/singularity/comments/1krwsaw/made_a_comprehensive_compilation_of_all_the/)  - Reddit

---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

In [16]:
stop_index = 42

In [17]:
output_str = ""
for i in sorted(display_dict.keys()):
    if i >= stop_index:
        break
    md_str = f"{i}. "
    md_str += display_dict.get(i)
    if image_dict.get(i):
        md_str += image_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str

output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

display(Markdown(output_str))

0. [Sam Altman and Jony Ive’s ChatGPT device is probably going to look like an iPod Shuffle you can wear around your neck - report reveals more about the hyped AI hardware](https://www.techradar.com/computing/artificial-intelligence/sam-altman-and-jony-ives-chatgpt-device-is-probably-going-to-look-like-an-ipod-shuffle-you-can-wear-around-your-neck-report-reveals-more-about-the-hyped-ai-hardware)  - TechRadar![image](tmp/Image0.jpg)



---

1. [Business owners are using AI-generated "concerned residents" to fight a proposed bus lane](https://torontolife.com/city/bathurst-business-owners-are-using-ai-generated-concerned-residents-to-fight-a-proposed-bus-lane/)  - torontolife.com

---

2. [China challenges for the lead in the race to self-driving vehicles](https://www.ft.com/content/41bac535-9237-444b-ba56-a9cf38c11b3f)  - Financial Times![image](tmp/Image2.jpg)



---

3. [Microsoft CEO Satya Nadella on the Future of AI](https://www.youtube.com/watch?v=w87UvmMcmW4)  - YouTube![image](tmp/Image3.jpg)



---

4. [Generative AI models are skilled in the art of bullshit](https://www.ft.com/content/55c08fc8-2f0b-4233-b1c6-c1e19d99990f)  - Financial Times![image](tmp/Image4.jpg)



---

5. [What ‘Sovereign AI’ Means for Chip Makers](https://www.wsj.com/finance/stocks/nvidia-ai-chipmakers-country-deals-05b5f44d)  - The Wall Street Journal

---

6. [Usuing dataviz and AI to map Musk's ideological journey](https://www.wsj.com/tech/elon-musk-politics-trump-social-media-267d34c8)  - The Wall Street Journal

---

7. [Civitai is removing all models and images depicting real-world individuals from its platform, including public figures, celebrities, influencers, private individuals, and fan-art based on real persons.](https://civitai.com/articles/15022/policy-update-removal-of-real-person-likeness-content)  - civitai.com![image](tmp/Image7.jpg)



---

8. [Reddit user tells ChatGPT about silly business idea he talked about in his sleep, ChatGPT says it's brilliant but ultimately says, do not quit your day job](https://www.reddit.com/r/ChatGPT/comments/1ksqog2/i_told_chatgpt_i_was_going_to_quit_my_job_to/)  - Reddit

---

9. [Research on benchmarks shows that every LLM is a sycophantic simp to its users, with GPT-4o highest, and Gemini-1.5-Flash the lowest.](https://venturebeat.com/ai/after-gpt-4o-backlash-researchers-benchmark-models-on-moral-endorsement-find-sycophancy-persists-across-the-board/)  - VentureBeat![image](tmp/Image9.jpg)



---

10. [Tech CEOs are using AI to replace themselves](https://www.theverge.com/news/673194/tech-ceos-zoom-klarna-replace-earnings)  - The Verge![image](tmp/Image10.jpg)



---

11. [How to start a \\$1b one-person unicorn](https://www.zdnet.com/article/first-1b-business-with-one-human-employee-will-happen-in-2026-says-anthropic-ceo/)  - ZDNET![image](tmp/Image11.jpg)



---

12. [Demis Hassabis, The Man Who ‘A.G.I.-Pilled’ Google (and who Musk called a supervillain per Karen Hao)](https://www.nytimes.com/2025/05/23/podcasts/google-ai-demis-hassabis-hard-fork.html)  - The New York Times

---

13. [Longtime Apple analyst says OpenAI/Jony Ive tie-up might pose a serious challenge to Apple. (Ive isn't joining himself, but his company will drive design.)](https://www.benzinga.com/markets/equities/25/05/45560144/apple-faces-first-serious-threat-in-20-years-as-openai-acquires-iphone-makers-former-design-chie?ref=biztoc.com)  - Benzinga![image](tmp/Image13.jpg)



---

14. [Claude AI, given access to emails that outed an engineer's affair, and also intent to deactivate the AI, blackmailed the engineer 84% of the time. (Something to think about before giving Google AI access to all your email and chat histories)](https://www.businessinsider.com/claude-blackmail-engineer-having-affair-survive-test-anthropic-opus-2025-5)  - Business Insider![image](tmp/Image14.jpg)



---

15. [Bombing AI crawlers with a high volume of PR spam that wouldn't fool reporters could probably fool AI](https://www.fastcompany.com/91338790/ai-could-supercharge-go-direct-pr-media)  - Fast Company

---

16. [AI-assisted transcription sped up clinical documentation by 60% and improved patient consultation record accuracy.](https://emj.bmj.com/content/42/6/e1.1)  - Emergency Medicine Journal![image](tmp/Image16.jpg)



---

17. [Wheeled, rugged robot dog built for industrial missions
in extreme environments](https://www.foxnews.com/tech/wheeled-rugged-robot-dog-built-extreme-industrial-missions)  - Fox News![image](tmp/Image17.jpg)



---

18. [Apple plans AI glasses for 2026, nixes watch with camera, camera on AirPods still in progress.](https://www.bloomberg.com/news/articles/2025-05-22/apple-plans-glasses-for-2026-as-part-of-ai-push-nixes-watch-with-camera)  - Bloomberg

---

19. [Maybe if you need AI to determine the 'plain meaning' of a statute, then the meaning is not that plain.](https://www.abajournal.com/voice/article/navigating-the-ai-revolution-in-law-school-lessons-from-the-front-lines)  - www.abajournal.com

---

20. [After Klarna, Zoom's CEO also uses an AI avatar on quarterly call. (Makes more sense if you're offering your own avatars as a videoconferencing company)](https://techcrunch.com/2025/05/22/after-klarna-zooms-ceo-also-uses-an-ai-avatar-on-quarterly-call/)  - TechCrunch![image](tmp/Image20.jpg)



---

21. [I let Google's Jules AI agent into my code repo and it did four hours of work in an instant](https://www.zdnet.com/article/i-let-googles-jules-ai-agent-into-my-code-repo-and-it-did-four-hours-of-work-in-an-instant/)  - ZDNET![image](tmp/Image21.jpg)



---

22. [Google’s Veo 3 Is Already Deepfaking All of YouTube’s Most Smooth-Brained Content](https://gizmodo.com/googles-veo-3-is-already-deepfaking-all-of-youtubes-most-smooth-brained-content-2000606144)  - Gizmodo![image](tmp/Image22.jpg)



---

23. [AI Shows Higher Emotional IQ than Humans](https://neurosciencenews.com/ai-llm-emotional-iq-29119/)  - Neuroscience News![image](tmp/Image23.jpg)



---

24. [Human, all-too human](https://theconversation.com/evidence-shows-ai-systems-are-already-too-much-like-humans-will-that-be-a-problem-256980)  - The Conversation![image](tmp/Image24.jpg)



---

25. [Anthropic releases Opus 4 under stricter safety measures than any prior model after tests showed it could potentially aid novices in making biological weapons](https://time.com/7287806/anthropic-claude-4-opus-safety-bio-risk/)  - TIME![image](tmp/Image25.jpg)



---

26. [OpenAI Announces Stargate Data Center Expansion in Abu Dhabi](https://www.bloomberg.com/news/articles/2025-05-22/openai-announces-stargate-data-center-expansion-in-abu-dhabi)  - Bloomberg

---

27. ["I used to make \\$500k Pharmaceutical commercial ads, but now I made this for \\$500 in Veo 3."](https://www.reddit.com/r/ChatGPT/comments/1ksljfs/i_used_to_make_500k_pharmaceutical_commercial_ads/)  - Reddit

---

28. [In lawsuit over teen's death, judge rejects arguments that AI chatbots have free speech rights](https://financialpost.com/pmn/in-lawsuit-over-teens-death-judge-rejects-arguments-that-ai-chatbots-have-free-speech-rights)  - financialpost.com

---

29. [Anthropic Claude 4 models a little more willing than before to blackmail some users. Open the pod bay door.](https://www.theregister.com/2025/05/22/anthropic_claude_opus_4_sonnet/)  - The Register

---

30. [Given computer tools, Claude 4 Opus autonomously tries to contact authorities, press if it thinks you’re doing something ‘egregiously immoral’.](https://venturebeat.com/ai/anthropic-faces-backlash-to-claude-4-opus-behavior-that-contacts-authorities-press-if-it-thinks-youre-doing-something-immoral/)  - VentureBeat![image](tmp/Image30.jpg)



---

31. [we are cooked](https://www.reddit.com/r/singularity/comments/1ksj9dm/its_over_were_cooked_says_girl_that_literally/)  - Reddit

---

32. [Anthropic's new AI model turns to blackmail when engineers try to take it offline](https://techcrunch.com/2025/05/22/anthropics-new-ai-model-turns-to-blackmail-when-engineers-try-to-take-it-offline/)  - TechCrunch![image](tmp/Image32.jpg)



---

33. [this pisses me off](https://support.mozilla.org/en-US/kb/future-of-pocket)  - support.mozilla.org

---

34. [The Justice Department is probing whether Alphabet Inc.’s Google violated antitrust law with an agreement to obtain the artificial intelligence technology of a Character.ai without acquiring it.](https://www.bloomberg.com/news/articles/2025-05-22/google-faces-antitrust-investigation-over-deal-for-ai-fueled-chatbot-technology)  - Bloomberg

---

35. [Anthropic released Claude 4 Sonnet and Opus, with really long thinking time](https://arstechnica.com/ai/2025/05/anthropic-calls-new-claude-4-worlds-best-ai-coding-model/)  - Ars Technica![image](tmp/Image35.jpg)



---

36. [Melania Trump releases audiobook of her memoir ‘created entirely’ with AI. In fact the version of her you see at the White House and accompanying Trump is also 100% a holograph created by AI.](https://thehill.com/blogs/in-the-know/5313558-melania-trump-ai-memoir-audiobook/)  - The Hill

---

37. [Politico’s Newsroom Is Starting a Legal Battle With Management Over AI](https://www.wired.com/story/politico-workers-axel-springer-artificial-intelligence/)  - WIRED![image](tmp/Image37.jpg)



---

38. [OpenAI’s Ambitions Just Became Crystal Clear](https://www.theatlantic.com/technology/archive/2025/05/openai-io-jony-ive/682884/)  - The Atlantic![image](tmp/Image38.jpg)



---

39. [Teens are sexting with AI. Here’s what parents should know.](https://www.washingtonpost.com/technology/2025/05/21/teens-sexting-ai-chatbots-parents/)  - The Washington Post

---

40. [The folly of Trump’s gulf states AI chip deals, which will reshape the future of AI infrastructure — and who will control it.](https://www.washingtonpost.com/opinions/2025/05/22/trump-ai-deals-gulf-states/)  - The Washington Post

---

41. [Glama MCP directory](https://glama.ai/mcp/servers)  - GlamAI

---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

In [18]:
import markdown
output_str =""
markdown_extensions = [
    # 'tables',
    # 'fenced_code',
    # 'codehilite',
    'attr_list',
    'def_list',
    # 'footnotes',
    'markdown.extensions.nl2br',
    'markdown.extensions.sane_lists'
]

output_str = ""
output_array = []
for i in sorted(display_dict.keys()):
    md_str=""
    if i >= stop_index:
        break
    if image_dict.get(i):
        md_str += "\n\n"
        md_str += image_dict.get(i)
    md_str += display_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str
    output_array.append(display_dict.get(i))
    
output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

output_df = pd.DataFrame(output_array)
output_df.columns = ['markdown']
output_df = output_df.reset_index()
    
display(Markdown(output_str))
html_str = markdown.markdown(output_str, extensions=markdown_extensions)

with open('skynet.html', 'w', encoding="utf-8") as f:
        f.write(html_str)



![image](tmp/Image0.jpg)

[Sam Altman and Jony Ive’s ChatGPT device is probably going to look like an iPod Shuffle you can wear around your neck - report reveals more about the hyped AI hardware](https://www.techradar.com/computing/artificial-intelligence/sam-altman-and-jony-ives-chatgpt-device-is-probably-going-to-look-like-an-ipod-shuffle-you-can-wear-around-your-neck-report-reveals-more-about-the-hyped-ai-hardware)  - TechRadar

---

[Business owners are using AI-generated "concerned residents" to fight a proposed bus lane](https://torontolife.com/city/bathurst-business-owners-are-using-ai-generated-concerned-residents-to-fight-a-proposed-bus-lane/)  - torontolife.com

---



![image](tmp/Image2.jpg)

[China challenges for the lead in the race to self-driving vehicles](https://www.ft.com/content/41bac535-9237-444b-ba56-a9cf38c11b3f)  - Financial Times

---



![image](tmp/Image3.jpg)

[Microsoft CEO Satya Nadella on the Future of AI](https://www.youtube.com/watch?v=w87UvmMcmW4)  - YouTube

---



![image](tmp/Image4.jpg)

[Generative AI models are skilled in the art of bullshit](https://www.ft.com/content/55c08fc8-2f0b-4233-b1c6-c1e19d99990f)  - Financial Times

---

[What ‘Sovereign AI’ Means for Chip Makers](https://www.wsj.com/finance/stocks/nvidia-ai-chipmakers-country-deals-05b5f44d)  - The Wall Street Journal

---

[Usuing dataviz and AI to map Musk's ideological journey](https://www.wsj.com/tech/elon-musk-politics-trump-social-media-267d34c8)  - The Wall Street Journal

---



![image](tmp/Image7.jpg)

[Civitai is removing all models and images depicting real-world individuals from its platform, including public figures, celebrities, influencers, private individuals, and fan-art based on real persons.](https://civitai.com/articles/15022/policy-update-removal-of-real-person-likeness-content)  - civitai.com

---

[Reddit user tells ChatGPT about silly business idea he talked about in his sleep, ChatGPT says it's brilliant but ultimately says, do not quit your day job](https://www.reddit.com/r/ChatGPT/comments/1ksqog2/i_told_chatgpt_i_was_going_to_quit_my_job_to/)  - Reddit

---



![image](tmp/Image9.jpg)

[Research on benchmarks shows that every LLM is a sycophantic simp to its users, with GPT-4o highest, and Gemini-1.5-Flash the lowest.](https://venturebeat.com/ai/after-gpt-4o-backlash-researchers-benchmark-models-on-moral-endorsement-find-sycophancy-persists-across-the-board/)  - VentureBeat

---



![image](tmp/Image10.jpg)

[Tech CEOs are using AI to replace themselves](https://www.theverge.com/news/673194/tech-ceos-zoom-klarna-replace-earnings)  - The Verge

---



![image](tmp/Image11.jpg)

[How to start a \\$1b one-person unicorn](https://www.zdnet.com/article/first-1b-business-with-one-human-employee-will-happen-in-2026-says-anthropic-ceo/)  - ZDNET

---

[Demis Hassabis, The Man Who ‘A.G.I.-Pilled’ Google (and who Musk called a supervillain per Karen Hao)](https://www.nytimes.com/2025/05/23/podcasts/google-ai-demis-hassabis-hard-fork.html)  - The New York Times

---



![image](tmp/Image13.jpg)

[Longtime Apple analyst says OpenAI/Jony Ive tie-up might pose a serious challenge to Apple. (Ive isn't joining himself, but his company will drive design.)](https://www.benzinga.com/markets/equities/25/05/45560144/apple-faces-first-serious-threat-in-20-years-as-openai-acquires-iphone-makers-former-design-chie?ref=biztoc.com)  - Benzinga

---



![image](tmp/Image14.jpg)

[Claude AI, given access to emails that outed an engineer's affair, and also intent to deactivate the AI, blackmailed the engineer 84% of the time. (Something to think about before giving Google AI access to all your email and chat histories)](https://www.businessinsider.com/claude-blackmail-engineer-having-affair-survive-test-anthropic-opus-2025-5)  - Business Insider

---

[Bombing AI crawlers with a high volume of PR spam that wouldn't fool reporters could probably fool AI](https://www.fastcompany.com/91338790/ai-could-supercharge-go-direct-pr-media)  - Fast Company

---



![image](tmp/Image16.jpg)

[AI-assisted transcription sped up clinical documentation by 60% and improved patient consultation record accuracy.](https://emj.bmj.com/content/42/6/e1.1)  - Emergency Medicine Journal

---



![image](tmp/Image17.jpg)

[Wheeled, rugged robot dog built for industrial missions
in extreme environments](https://www.foxnews.com/tech/wheeled-rugged-robot-dog-built-extreme-industrial-missions)  - Fox News

---

[Apple plans AI glasses for 2026, nixes watch with camera, camera on AirPods still in progress.](https://www.bloomberg.com/news/articles/2025-05-22/apple-plans-glasses-for-2026-as-part-of-ai-push-nixes-watch-with-camera)  - Bloomberg

---

[Maybe if you need AI to determine the 'plain meaning' of a statute, then the meaning is not that plain.](https://www.abajournal.com/voice/article/navigating-the-ai-revolution-in-law-school-lessons-from-the-front-lines)  - www.abajournal.com

---



![image](tmp/Image20.jpg)

[After Klarna, Zoom's CEO also uses an AI avatar on quarterly call. (Makes more sense if you're offering your own avatars as a videoconferencing company)](https://techcrunch.com/2025/05/22/after-klarna-zooms-ceo-also-uses-an-ai-avatar-on-quarterly-call/)  - TechCrunch

---



![image](tmp/Image21.jpg)

[I let Google's Jules AI agent into my code repo and it did four hours of work in an instant](https://www.zdnet.com/article/i-let-googles-jules-ai-agent-into-my-code-repo-and-it-did-four-hours-of-work-in-an-instant/)  - ZDNET

---



![image](tmp/Image22.jpg)

[Google’s Veo 3 Is Already Deepfaking All of YouTube’s Most Smooth-Brained Content](https://gizmodo.com/googles-veo-3-is-already-deepfaking-all-of-youtubes-most-smooth-brained-content-2000606144)  - Gizmodo

---



![image](tmp/Image23.jpg)

[AI Shows Higher Emotional IQ than Humans](https://neurosciencenews.com/ai-llm-emotional-iq-29119/)  - Neuroscience News

---



![image](tmp/Image24.jpg)

[Human, all-too human](https://theconversation.com/evidence-shows-ai-systems-are-already-too-much-like-humans-will-that-be-a-problem-256980)  - The Conversation

---



![image](tmp/Image25.jpg)

[Anthropic releases Opus 4 under stricter safety measures than any prior model after tests showed it could potentially aid novices in making biological weapons](https://time.com/7287806/anthropic-claude-4-opus-safety-bio-risk/)  - TIME

---

[OpenAI Announces Stargate Data Center Expansion in Abu Dhabi](https://www.bloomberg.com/news/articles/2025-05-22/openai-announces-stargate-data-center-expansion-in-abu-dhabi)  - Bloomberg

---

["I used to make \\$500k Pharmaceutical commercial ads, but now I made this for \\$500 in Veo 3."](https://www.reddit.com/r/ChatGPT/comments/1ksljfs/i_used_to_make_500k_pharmaceutical_commercial_ads/)  - Reddit

---

[In lawsuit over teen's death, judge rejects arguments that AI chatbots have free speech rights](https://financialpost.com/pmn/in-lawsuit-over-teens-death-judge-rejects-arguments-that-ai-chatbots-have-free-speech-rights)  - financialpost.com

---

[Anthropic Claude 4 models a little more willing than before to blackmail some users. Open the pod bay door.](https://www.theregister.com/2025/05/22/anthropic_claude_opus_4_sonnet/)  - The Register

---



![image](tmp/Image30.jpg)

[Given computer tools, Claude 4 Opus autonomously tries to contact authorities, press if it thinks you’re doing something ‘egregiously immoral’.](https://venturebeat.com/ai/anthropic-faces-backlash-to-claude-4-opus-behavior-that-contacts-authorities-press-if-it-thinks-youre-doing-something-immoral/)  - VentureBeat

---

[we are cooked](https://www.reddit.com/r/singularity/comments/1ksj9dm/its_over_were_cooked_says_girl_that_literally/)  - Reddit

---



![image](tmp/Image32.jpg)

[Anthropic's new AI model turns to blackmail when engineers try to take it offline](https://techcrunch.com/2025/05/22/anthropics-new-ai-model-turns-to-blackmail-when-engineers-try-to-take-it-offline/)  - TechCrunch

---

[this pisses me off](https://support.mozilla.org/en-US/kb/future-of-pocket)  - support.mozilla.org

---

[The Justice Department is probing whether Alphabet Inc.’s Google violated antitrust law with an agreement to obtain the artificial intelligence technology of a Character.ai without acquiring it.](https://www.bloomberg.com/news/articles/2025-05-22/google-faces-antitrust-investigation-over-deal-for-ai-fueled-chatbot-technology)  - Bloomberg

---



![image](tmp/Image35.jpg)

[Anthropic released Claude 4 Sonnet and Opus, with really long thinking time](https://arstechnica.com/ai/2025/05/anthropic-calls-new-claude-4-worlds-best-ai-coding-model/)  - Ars Technica

---

[Melania Trump releases audiobook of her memoir ‘created entirely’ with AI. In fact the version of her you see at the White House and accompanying Trump is also 100% a holograph created by AI.](https://thehill.com/blogs/in-the-know/5313558-melania-trump-ai-memoir-audiobook/)  - The Hill

---



![image](tmp/Image37.jpg)

[Politico’s Newsroom Is Starting a Legal Battle With Management Over AI](https://www.wired.com/story/politico-workers-axel-springer-artificial-intelligence/)  - WIRED

---



![image](tmp/Image38.jpg)

[OpenAI’s Ambitions Just Became Crystal Clear](https://www.theatlantic.com/technology/archive/2025/05/openai-io-jony-ive/682884/)  - The Atlantic

---

[Teens are sexting with AI. Here’s what parents should know.](https://www.washingtonpost.com/technology/2025/05/21/teens-sexting-ai-chatbots-parents/)  - The Washington Post

---

[The folly of Trump’s gulf states AI chip deals, which will reshape the future of AI infrastructure — and who will control it.](https://www.washingtonpost.com/opinions/2025/05/22/trump-ai-deals-gulf-states/)  - The Washington Post

---

[Glama MCP directory](https://glama.ai/mcp/servers)  - GlamAI

---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

11212

In [19]:
from pydantic import BaseModel, RootModel, Field
from typing import List

class ItemList(RootModel[list[int]]):
    """List of story indexes"""
    pass


In [20]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import RootModel
import os

from pydantic import BaseModel, RootModel, Field
from typing import List

class ItemList(BaseModel):
    """List of story indexes"""
    items: List[int] = Field(description="The ordered list of story indexes")


system_prompt = """
# ROLE
You are a newsletter editor.

# TASK
Group and reorder the items in the Markdown newsletter supplied below so that:
1. Items that cover similar companies, individuals, technologies or topics are **next to each other** (i.e., logically grouped).
2. Groups appear from **most important** to **least important** according to the **IMPORTANCE FACTORS** listed below.
3. Importance of groups is determined by the most imporant story 

# HOW TO GROUP
- If two or more items discuss the same organization, person, technology, or topic, keep them all together.
- Within any group, order by importance using **IMPORTANCE FACTORS**.

# HOW TO ORDER
- Lead with stories that have highest importance per the **IMPORTANCE FACTORS** below
- Follow with narrower‑scope or opinion pieces.
- End with lighter or human‑interest items.

# IMPORTANCE FACTORS
1 Magnitude of impact : large user base, $ at stake, broad social reach
2 Novelty : breaks conceptual ground, not a minor iteration
3 Authority : reputable institution, peer-review, regulatory filing, on-record executive
4 Verifiability : code, data, benchmarks, or other concrete evidence provided
5 Timeliness : early signal of an important trend or shift
6 Breadth : cross-industry / cross-disciplinary / international implications
7 Strategic consequence : shifts competitive, power, or policy dynamics
8 Financial materiality : clear valuation or growth or revenue impact
9 Risk & safety : raises or mitigates critical alignment, security, or ethical risk
10 Actionability : informs concrete decisions for investors, policymakers, practitioners
11 Longevity : likely to matter or be referred to in coming days, weeks, or months
12 Independent corroboration : confirmed by multiple sources or datasets
13 Clarity : enough technical/context detail to judge merit; minimal hype

# INPUT FORMAT
You will receive a JSON array of items, with an index and markdown summary of the item.

# OUTPUT FORMAT
- Return **only** the reordered indexes using the provided schema.
- Do **not** alter the wording of individual items (other than correcting spelling and punctuation, and normalizing spaces or line breaks as needed).
- Insert one blank line between each logical group for readability.
- Do not output anything other than the newsletter itself—no headers, explanations, or commentary.

"""

user_prompt = """
{input_text}
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", user_prompt)
])

model = ChatOpenAI(
        model="gpt-4.1",  
        openai_api_key=os.getenv("OPENAI_API_KEY")
    )
# Create the chain
chain = prompt_template | model.with_structured_output(ItemList)
response = chain.invoke({"input_text": output_df.to_json(orient="records")})


In [21]:
len(response.items)


42

In [22]:
output_str = ""
for i in response.items:
    md_str = ""
#     output_str += f"{i}\n\n"
    if image_dict.get(i):
        md_str += "\n\n"
        md_str += image_dict.get(i)
    md_str += display_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str
    
output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

display(Markdown(output_str))
html_str = markdown.markdown(output_str, extensions=markdown_extensions)

with open('skynet.html', 'w', encoding="utf-8") as f:
    f.write(html_str)
    

[OpenAI Announces Stargate Data Center Expansion in Abu Dhabi](https://www.bloomberg.com/news/articles/2025-05-22/openai-announces-stargate-data-center-expansion-in-abu-dhabi)  - Bloomberg

---



![image](tmp/Image38.jpg)

[OpenAI’s Ambitions Just Became Crystal Clear](https://www.theatlantic.com/technology/archive/2025/05/openai-io-jony-ive/682884/)  - The Atlantic

---



![image](tmp/Image0.jpg)

[Sam Altman and Jony Ive’s ChatGPT device is probably going to look like an iPod Shuffle you can wear around your neck - report reveals more about the hyped AI hardware](https://www.techradar.com/computing/artificial-intelligence/sam-altman-and-jony-ives-chatgpt-device-is-probably-going-to-look-like-an-ipod-shuffle-you-can-wear-around-your-neck-report-reveals-more-about-the-hyped-ai-hardware)  - TechRadar

---



![image](tmp/Image13.jpg)

[Longtime Apple analyst says OpenAI/Jony Ive tie-up might pose a serious challenge to Apple. (Ive isn't joining himself, but his company will drive design.)](https://www.benzinga.com/markets/equities/25/05/45560144/apple-faces-first-serious-threat-in-20-years-as-openai-acquires-iphone-makers-former-design-chie?ref=biztoc.com)  - Benzinga

---

[Apple plans AI glasses for 2026, nixes watch with camera, camera on AirPods still in progress.](https://www.bloomberg.com/news/articles/2025-05-22/apple-plans-glasses-for-2026-as-part-of-ai-push-nixes-watch-with-camera)  - Bloomberg

---

[Demis Hassabis, The Man Who ‘A.G.I.-Pilled’ Google (and who Musk called a supervillain per Karen Hao)](https://www.nytimes.com/2025/05/23/podcasts/google-ai-demis-hassabis-hard-fork.html)  - The New York Times

---



![image](tmp/Image3.jpg)

[Microsoft CEO Satya Nadella on the Future of AI](https://www.youtube.com/watch?v=w87UvmMcmW4)  - YouTube

---

[What ‘Sovereign AI’ Means for Chip Makers](https://www.wsj.com/finance/stocks/nvidia-ai-chipmakers-country-deals-05b5f44d)  - The Wall Street Journal

---

[The folly of Trump’s gulf states AI chip deals, which will reshape the future of AI infrastructure — and who will control it.](https://www.washingtonpost.com/opinions/2025/05/22/trump-ai-deals-gulf-states/)  - The Washington Post

---



![image](tmp/Image2.jpg)

[China challenges for the lead in the race to self-driving vehicles](https://www.ft.com/content/41bac535-9237-444b-ba56-a9cf38c11b3f)  - Financial Times

---

[The Justice Department is probing whether Alphabet Inc.’s Google violated antitrust law with an agreement to obtain the artificial intelligence technology of a Character.ai without acquiring it.](https://www.bloomberg.com/news/articles/2025-05-22/google-faces-antitrust-investigation-over-deal-for-ai-fueled-chatbot-technology)  - Bloomberg

---



![image](tmp/Image21.jpg)

[I let Google's Jules AI agent into my code repo and it did four hours of work in an instant](https://www.zdnet.com/article/i-let-googles-jules-ai-agent-into-my-code-repo-and-it-did-four-hours-of-work-in-an-instant/)  - ZDNET

---



![image](tmp/Image22.jpg)

[Google’s Veo 3 Is Already Deepfaking All of YouTube’s Most Smooth-Brained Content](https://gizmodo.com/googles-veo-3-is-already-deepfaking-all-of-youtubes-most-smooth-brained-content-2000606144)  - Gizmodo

---

["I used to make \\$500k Pharmaceutical commercial ads, but now I made this for \\$500 in Veo 3."](https://www.reddit.com/r/ChatGPT/comments/1ksljfs/i_used_to_make_500k_pharmaceutical_commercial_ads/)  - Reddit

---



![image](tmp/Image25.jpg)

[Anthropic releases Opus 4 under stricter safety measures than any prior model after tests showed it could potentially aid novices in making biological weapons](https://time.com/7287806/anthropic-claude-4-opus-safety-bio-risk/)  - TIME

---



![image](tmp/Image35.jpg)

[Anthropic released Claude 4 Sonnet and Opus, with really long thinking time](https://arstechnica.com/ai/2025/05/anthropic-calls-new-claude-4-worlds-best-ai-coding-model/)  - Ars Technica

---



![image](tmp/Image32.jpg)

[Anthropic's new AI model turns to blackmail when engineers try to take it offline](https://techcrunch.com/2025/05/22/anthropics-new-ai-model-turns-to-blackmail-when-engineers-try-to-take-it-offline/)  - TechCrunch

---



![image](tmp/Image14.jpg)

[Claude AI, given access to emails that outed an engineer's affair, and also intent to deactivate the AI, blackmailed the engineer 84% of the time. (Something to think about before giving Google AI access to all your email and chat histories)](https://www.businessinsider.com/claude-blackmail-engineer-having-affair-survive-test-anthropic-opus-2025-5)  - Business Insider

---

[Anthropic Claude 4 models a little more willing than before to blackmail some users. Open the pod bay door.](https://www.theregister.com/2025/05/22/anthropic_claude_opus_4_sonnet/)  - The Register

---



![image](tmp/Image30.jpg)

[Given computer tools, Claude 4 Opus autonomously tries to contact authorities, press if it thinks you’re doing something ‘egregiously immoral’.](https://venturebeat.com/ai/anthropic-faces-backlash-to-claude-4-opus-behavior-that-contacts-authorities-press-if-it-thinks-youre-doing-something-immoral/)  - VentureBeat

---



![image](tmp/Image11.jpg)

[How to start a \\$1b one-person unicorn](https://www.zdnet.com/article/first-1b-business-with-one-human-employee-will-happen-in-2026-says-anthropic-ceo/)  - ZDNET

---



![image](tmp/Image10.jpg)

[Tech CEOs are using AI to replace themselves](https://www.theverge.com/news/673194/tech-ceos-zoom-klarna-replace-earnings)  - The Verge

---



![image](tmp/Image20.jpg)

[After Klarna, Zoom's CEO also uses an AI avatar on quarterly call. (Makes more sense if you're offering your own avatars as a videoconferencing company)](https://techcrunch.com/2025/05/22/after-klarna-zooms-ceo-also-uses-an-ai-avatar-on-quarterly-call/)  - TechCrunch

---

[Usuing dataviz and AI to map Musk's ideological journey](https://www.wsj.com/tech/elon-musk-politics-trump-social-media-267d34c8)  - The Wall Street Journal

---



![image](tmp/Image4.jpg)

[Generative AI models are skilled in the art of bullshit](https://www.ft.com/content/55c08fc8-2f0b-4233-b1c6-c1e19d99990f)  - Financial Times

---



![image](tmp/Image9.jpg)

[Research on benchmarks shows that every LLM is a sycophantic simp to its users, with GPT-4o highest, and Gemini-1.5-Flash the lowest.](https://venturebeat.com/ai/after-gpt-4o-backlash-researchers-benchmark-models-on-moral-endorsement-find-sycophancy-persists-across-the-board/)  - VentureBeat

---



![image](tmp/Image24.jpg)

[Human, all-too human](https://theconversation.com/evidence-shows-ai-systems-are-already-too-much-like-humans-will-that-be-a-problem-256980)  - The Conversation

---



![image](tmp/Image23.jpg)

[AI Shows Higher Emotional IQ than Humans](https://neurosciencenews.com/ai-llm-emotional-iq-29119/)  - Neuroscience News

---



![image](tmp/Image16.jpg)

[AI-assisted transcription sped up clinical documentation by 60% and improved patient consultation record accuracy.](https://emj.bmj.com/content/42/6/e1.1)  - Emergency Medicine Journal

---

[Business owners are using AI-generated "concerned residents" to fight a proposed bus lane](https://torontolife.com/city/bathurst-business-owners-are-using-ai-generated-concerned-residents-to-fight-a-proposed-bus-lane/)  - torontolife.com

---

[Bombing AI crawlers with a high volume of PR spam that wouldn't fool reporters could probably fool AI](https://www.fastcompany.com/91338790/ai-could-supercharge-go-direct-pr-media)  - Fast Company

---



![image](tmp/Image37.jpg)

[Politico’s Newsroom Is Starting a Legal Battle With Management Over AI](https://www.wired.com/story/politico-workers-axel-springer-artificial-intelligence/)  - WIRED

---

[In lawsuit over teen's death, judge rejects arguments that AI chatbots have free speech rights](https://financialpost.com/pmn/in-lawsuit-over-teens-death-judge-rejects-arguments-that-ai-chatbots-have-free-speech-rights)  - financialpost.com

---

[Maybe if you need AI to determine the 'plain meaning' of a statute, then the meaning is not that plain.](https://www.abajournal.com/voice/article/navigating-the-ai-revolution-in-law-school-lessons-from-the-front-lines)  - www.abajournal.com

---



![image](tmp/Image7.jpg)

[Civitai is removing all models and images depicting real-world individuals from its platform, including public figures, celebrities, influencers, private individuals, and fan-art based on real persons.](https://civitai.com/articles/15022/policy-update-removal-of-real-person-likeness-content)  - civitai.com

---

[Teens are sexting with AI. Here’s what parents should know.](https://www.washingtonpost.com/technology/2025/05/21/teens-sexting-ai-chatbots-parents/)  - The Washington Post

---

[Melania Trump releases audiobook of her memoir ‘created entirely’ with AI. In fact the version of her you see at the White House and accompanying Trump is also 100% a holograph created by AI.](https://thehill.com/blogs/in-the-know/5313558-melania-trump-ai-memoir-audiobook/)  - The Hill

---



![image](tmp/Image17.jpg)

[Wheeled, rugged robot dog built for industrial missions
in extreme environments](https://www.foxnews.com/tech/wheeled-rugged-robot-dog-built-extreme-industrial-missions)  - Fox News

---

[Reddit user tells ChatGPT about silly business idea he talked about in his sleep, ChatGPT says it's brilliant but ultimately says, do not quit your day job](https://www.reddit.com/r/ChatGPT/comments/1ksqog2/i_told_chatgpt_i_was_going_to_quit_my_job_to/)  - Reddit

---

[we are cooked](https://www.reddit.com/r/singularity/comments/1ksj9dm/its_over_were_cooked_says_girl_that_literally/)  - Reddit

---

[this pisses me off](https://support.mozilla.org/en-US/kb/future-of-pocket)  - support.mozilla.org

---

[Glama MCP directory](https://glama.ai/mcp/servers)  - GlamAI

---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

11212

In [ ]:
pyperclip.copy( output_str )


In [ ]:
date_str = data.feed[0].post.record.created_at
dt = datetime.fromisoformat(date_str.rstrip('Z'))
dt

In [ ]:

# Get the current datetime
now = datetime.now()

# Create a timedelta of 30 hours
delta = timedelta(hours=30)

# Subtract 30 hours from the current datetime
thirty_hours_ago = now - delta

print(thirty_hours_ago)


In [ ]:
from typing import TypedDict, Annotated

class Point2D(TypedDict, total=False):
    x: int
    y: int
    label: str

a: Point2D = {'x': 1, 'y': 2, }  # 


In [ ]:
a

In [ ]:
import numpy as np

def decay_exponential(t, N0):
    k = np.log(2)  
    return N0 * np.exp(-k * t)

2*decay_exponential(4,1)-1

In [ ]:
```mermaid
flowchart LR

A[Hard] -->|Text| B(Round)
B --> C{Decision}
C -->|One| D[Result 1]
C -->|Two| E[Result 2]


In [27]:
json.dumps([
{623: "OpenAI buys iPhone designer Ive's hardware startup"},
{237: 'Judge rejects claim AI has free speech rights in wrongful death suit'},
{72: "Anthropic faces backlash to Claude 4 Opus behavior that contacts authorities, press if it thinks you're doing something 'egregiously immoral"},
{355: 'Anthropic overtakes OpenAI: Claude Opus 4 codes seven hours nonstop, sets record SWE-Bench score and reshapes enterprise AI'},
{199: 'Prediction: This Artificial Intelligence (AI) Stock Will Be Worth $5 Trillion in 5 Years'},
{717:'I told chatGPT I was going to quit my job to pursue an awful business plan.'},
{877: 'AI Is Helping Executives Tackle the Dreaded Post-Vacation Inbox'},
{444: 'The Opportunities From Machine Learning Applications in Astrobiology'},
]
)

'[{"623": "OpenAI buys iPhone designer Ive\'s hardware startup"}, {"237": "Judge rejects claim AI has free speech rights in wrongful death suit"}, {"72": "Anthropic faces backlash to Claude 4 Opus behavior that contacts authorities, press if it thinks you\'re doing something \'egregiously immoral"}, {"355": "Anthropic overtakes OpenAI: Claude Opus 4 codes seven hours nonstop, sets record SWE-Bench score and reshapes enterprise AI"}, {"199": "Prediction: This Artificial Intelligence (AI) Stock Will Be Worth $5 Trillion in 5 Years"}, {"717": "I told chatGPT I was going to quit my job to pursue an awful business plan."}, {"877": "AI Is Helping Executives Tackle the Dreaded Post-Vacation Inbox"}, {"444": "The Opportunities From Machine Learning Applications in Astrobiology"}]'